<a href="https://colab.research.google.com/github/MrPrabhathPeri/Stocks-Predictor/blob/main/Stocks_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_squared_error
from math import sqrt


In [2]:
# Load the dataset
from google.colab import files
uploaded = files.upload()

# Assuming the dataset is a CSV file
df = pd.read_csv(list(uploaded.keys())[0])

# Display the first few rows of the dataset
df.head()


Saving adobe_stock_data.csv to adobe_stock_data (1).csv


,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-04,36.650002,37.299999,36.650002,37.090000,37.090000,4710200
1,2010-01-05,37.040001,37.799999,36.869999,37.700001,37.700001,7108800
2,2010-01-06,37.330002,37.740002,37.200001,37.619999,37.619999,5336400
3,2010-01-07,37.410000,37.590000,36.810001,36.889999,36.889999,5576700
4,2010-01-08,36.750000,36.970001,36.340000,36.689999,36.689999,5429200


In [3]:
# Convert the Date column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Sort the data by date
df = df.sort_values('Date')

# Use the 'Close' price for prediction
data = df['Close'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Define the number of previous days to consider for prediction
n_past = 60
n_future = 1

X = []
y = []

for i in range(n_past, len(scaled_data) - n_future + 1):
    X.append(scaled_data[i - n_past:i, 0])
    y.append(scaled_data[i + n_future - 1:i + n_future, 0])

X, y = np.array(X), np.array(y)

# Reshape X to be 3-dimensional (samples, timesteps, features)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))


In [4]:
model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [5]:
# Train the model
model.fit(X, y, batch_size=1, epochs=1)


3503/3503 ━━━━━━━━━━━━━━━━━━━━ 122s 34ms/step - loss: 0.0018


In [7]:
# Prepare test data (using the last 60 days from the dataset)
test_data = scaled_data[-n_past:]

X_test = []

for i in range(n_past, len(test_data)):
    X_test.append(test_data[i - n_past:i, 0])

if len(X_test) > 0:
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # Make predictions
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)

    # Plot the results
    plt.figure(figsize=(10,6))
    plt.plot(df['Date'][-len(predictions):], data[-len(predictions):], color='blue', label='Actual Stock Price')
    plt.plot(df['Date'][-len(predictions):], predictions, color='red', label='Predicted Stock Price')
    plt.title('Adobe Stock Price Prediction')
    plt.xlabel('Date')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.show()
else:
    print("Insufficient data for making predictions. Please check your dataset and try again.")


Insufficient data for making predictions. Please check your dataset and try again.


In [10]:
# Check if predictions were made
if 'predictions' in locals():
    # Calculate the RMSE
    rmse = sqrt(mean_squared_error(data[-len(predictions):], predictions))
    print(f'Root Mean Squared Error: {rmse}')
else:
    print("Predictions were not made due to insufficient data. Skipping RMSE calculation.")


Predictions were not made due to insufficient data. Skipping RMSE calculation.


In [11]:
model.save('adobe_stock_predictor.h5')


In [12]:
from tensorflow.keras.models import load_model

model = load_model('adobe_stock_predictor.h5')


In [19]:
new_data = [
    36.88999938964844, 36.689998626708984, 36.209999084472656, 35.65999984741211,
    36.279998779296875, 35.900001525878906, 35.869998931884766, 36.95000076293945,
    36.4900016784668, 35.81999969482422, 36.88999938964844, 36.689998626708984,
    36.209999084472656, 35.65999984741211, 36.279998779296875, 35.900001525878906,
    35.869998931884766, 36.95000076293945, 36.4900016784668, 35.81999969482422,
    36.88999938964844, 36.689998626708984, 36.209999084472656, 35.65999984741211,
    36.279998779296875, 35.900001525878906, 35.869998931884766, 36.95000076293945,
    36.4900016784668, 35.81999969482422, 36.88999938964844, 36.689998626708984,
    36.209999084472656, 35.65999984741211, 36.279998779296875, 35.900001525878906,
    35.869998931884766, 36.95000076293945, 36.4900016784668, 35.81999969482422,
    36.88999938964844, 36.689998626708984, 36.209999084472656, 35.65999984741211,
    36.279998779296875, 35.900001525878906, 35.869998931884766, 36.95000076293945,
    36.4900016784668, 35.81999969482422, 36.88999938964844, 36.689998626708984,
    36.209999084472656, 35.65999984741211, 36.279998779296875, 35.900001525878906,
    35.869998931884766, 36.95000076293945, 36.4900016784668, 35.81999969482422
]


In [20]:
new_data = np.array(new_data).reshape(-1, 1)
new_data_scaled = scaler.transform(new_data)
new_data_scaled = np.reshape(new_data_scaled, (1, n_past, 1))  # Assuming n_past is 60


In [21]:
predicted_price_scaled = model.predict(new_data_scaled)
predicted_price = scaler.inverse_transform(predicted_price_scaled)
print(f"Predicted next day's closing price: {predicted_price[0][0]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted next day's closing price: 44.28889846801758
